## Задача 4

### Решить интегральное уравнение Фредгольма второго рода (задание - 4.2.2, стр.38, варианты - 4.3, стр.39, по второй части методички).

### Вариант 7

$u(x) - 0.5 \int_0^1 th(xy) u(y)\,dy = x - 0.5$

$\epsilon = 0.000001$

$H(x, y) = th(xy)$

$f(x) = x - 0.5$

$[a, b] = [0, 1]$

Составная формула Симпсона.

Входные данные

In [4]:
import maths
import numpy as np
import math
from pandas import DataFrame

def H(x, y):
    return math.tanh(x * y)

def f(x):
    return x - 0.5

a, b = 0.0, 1.0

eps = 0.000001
n = 2
m = 1

Метод механических квадратур

In [5]:
def mechanical_quadrature_method(H, f, n: int, a: float, b: float):
    D = np.zeros((n + 1, n + 1), dtype=float)
    (coeffs, points) = maths.make_simpson_coefs(a, b, n)
    for j in range(n + 1):
        for k in range(n + 1):
            D[j, k] = maths.kronecker_symbol(j, k) - coeffs[k] * H(coeffs[j], coeffs[k])

    g = np.array([f(points[k]) for k in range(n + 1)])
    z = np.linalg.solve(D, g)

    return coeffs, points, z

def solve_in_point(x: float, coeffs, points, z, H, f):
    n_plus_one = len(coeffs)
    ans = f(x)
    for i in range(n_plus_one):
        ans += coeffs[i] * H(x, points[i]) * z[i]
    return ans

Решение задачи с поставленной точностью и вывод таблицы

In [6]:
coeffs, points, z = mechanical_quadrature_method(H, f, n * m, a, b)
current_ans = np.array([solve_in_point(a, coeffs, points, z, H, f),
                        solve_in_point((a + b) / 2, coeffs, points, z, H, f),
                        solve_in_point(b, coeffs, points, z, H, f)])

error = None
ans_table = [[f"u^{n*m}(x)", current_ans[0], current_ans[1], current_ans[2]]]
while error is None or error > eps:
    prev_ans = current_ans.copy()
    m *= 2
    coeffs, points, z = mechanical_quadrature_method(H, f, n * m, a, b)
    current_ans = np.array([solve_in_point(a, coeffs, points, z, H, f),
                            solve_in_point((a + b) / 2, coeffs, points, z, H, f),
                            solve_in_point(b, coeffs, points, z, H, f)])
    error = np.abs(current_ans - prev_ans).max()
    ans_table.append([f"u^{n*m}(x)", current_ans[0], current_ans[1], current_ans[2]])

last_err = current_ans - prev_ans
ans_table.append([f"u^{m * n}(x) - u^{m * n // 2}(x)", last_err[0], last_err[1], last_err[2]])
df = DataFrame(ans_table, columns=["x", "a", "(a + b) / 2", "b"])
df

,x,a,(a + b) / 2,b
0,u^2(x),-0.5,3.850976e-02,5.634662e-01
1,u^4(x),-0.5,3.875525e-02,5.642523e-01
2,u^8(x),-0.5,3.876983e-02,5.643068e-01
3,u^16(x),-0.5,3.877073e-02,5.643101e-01
4,u^32(x),-0.5,3.877079e-02,5.643103e-01
5,u^32(x) - u^16(x),0.0,5.612160e-08,2.076202e-07
